In [ ]:
#_filter2_magics_i1
##%overwritefile
##%file:../src/_filter2_magics_i1.py
##%noruncode
for pkey,pvalue in self.IBplugins.items():
    # print( pkey +":"+str(len(pvalue))+"\n")
    for pobj in pvalue:
        newline=''
        try:
            if line.strip()[3:] in pobj.getIDBptag(pobj):
                newline=pobj.on_IBpCodescanning(pobj,magics,line)
                if newline=='':continue

        except Exception as e:
            pass
        finally:pass
        # if newline!=None and newline!='':
        #     actualCode += newline + '\n'
##

In [ ]:
#_filter2_magics_i2
##%overwritefile
##%file:../src/_filter2_magics_i2.py
##%noruncode

for pkey,pvalue in self.ISplugins.items():
    # print( pkey +":"+str(len(pvalue))+"\n")
    for pobj in pvalue:
        newline=''
        try:
            if key in pobj.getIDSptag(pobj):
                newline=pobj.on_ISpCodescanning(pobj,key,value,magics,line)
                if newline=='':continue

        except Exception as e:
            pass
        finally:pass
        if newline!=None and newline!='':
            actualCode += newline + '\n'
# always add empty line, so line numbers don't change
# actualCode += '\n'

In [ ]:
#_filter2_magics_pend
##%overwritefile
##%file:../src/_filter2_magics_pend.py
##%noruncode

if len(self.addkey2dict(magics,'file'))>0 :
    newactualCode=''
    for line in actualCode.splitlines():
        try:
            # if len(self.addkey2dict(magics,'test'))<1:
            line=self.kobj.cleantestcode(line)
            if line=='':continue
            ##调用双标签接口
            line=self.kobj.callIDplugin(line)
            ##
            if line=='':continue
            line=self.kobj.cleandqm(line)
            if line=='':continue
            line=self.kobj.cleansqm(line)
            if self.kobj.cleannotes(line)=='':
                continue
            else:
                newactualCode += line + '\n'
        except Exception as e:
            self.kobj._log(str(e),3)

In [ ]:
##触发接口调用
##%overwritefile
##%file:../src/_callICodescan.py
##%noruncode
def raise_ICodescan(self,magics,code)->Tuple[bool,str]:
    bcancel_exec=False
    bretcancel_exec=False
    newcode=code
    # for pluginlist in self.plugins:
    for pkey,pvalue in self.ICodePreprocs.items():
        # print( pkey +":"+str(len(pvalue))+"\n")
        for pobj in pvalue:
            try:
                bretcancel_exec,newcode=pobj.on_Codescanning(pobj,magics,newcode)
                bcancel_exec=bretcancel_exec & bcancel_exec
                if bcancel_exec:
                    return bcancel_exec,newcode
            except Exception as e:
                self.kobj._logln(pobj.getName(pobj)+"---"+str(e))
            finally:pass
    return bcancel_exec,newcode
    


In [4]:
#_filter2_magics
##%overwritefile
##%file:../src/_filter2_magics.py
##%file:../../jupyter-MyPython-kernel/plugins/_filter2_magics.py
##%file:../../jupyter-MyC-kernel/plugins/_filter2_magics.py
##%file:../../jupyter-MyDart-kernel/plugins/_filter2_magics.py

##%file:../../jupyter-MyKotlin-kernel/plugins/_filter2_magics.py
##%file:../../jupyter-MyGroovy-kernel/plugins/_filter2_magics.py
##%file:../../jupyter-MyJava-kernel/plugins/_filter2_magics.py

##%file:../../jupyter-MyGjs-kernel/plugins/_filter2_magics.py
##%file:../../jupyter-MyVala-kernel/plugins/_filter2_magics.py
##%file:../../jupyter-MyNodejs-kernel/plugins/_filter2_magics.py

##%file:../../jupyter-MyBash-kernel/plugins/_filter2_magics.py

##%noruncode
##########################
from math import exp
from queue import Queue
from threading import Thread

from ipykernel.kernelbase import Kernel
from pexpect import replwrap, EOF
from jinja2 import Environment, PackageLoader, select_autoescape,Template
from abc import ABCMeta, abstractmethod
from typing import List, Dict, Tuple, Sequence
from shutil import copyfile
from plugins.ISpecialID import IStag,IDtag,IBtag,ITag
import pexpect
import signal
import typing 
import typing as t
import re
import signal
import subprocess
import tempfile
import os
import sys
import traceback
import os.path as path
import codecs
import time
import importlib
import importlib.util
import inspect

###############################
class Magics():
    plugins=None
    ISplugins=None
    IDplugins=None
    IBplugins=None
    kobj=None
    
    plugins=[ISplugins,IDplugins,IBplugins]
    def __init__(self,kobj,plugins:List,ICodePreprocs):
        self.kobj=kobj
        self.plugins=plugins
        self.ICodePreprocs=ICodePreprocs
        self.ISplugins=self.plugins[0]
        self.IDplugins=self.plugins[1]
        self.IBplugins=self.plugins[2]
        pass
    ##%include:../src/_is_specialID.py
    def addkey2dict(self,magics:Dict,key:str):
        if not magics.__contains__(key):
            d={key:[]}
            magics.update(d)
        return magics[key]
    
    ##%include:../src/_callICodescan.py
    def filter(self, code):
        ##魔法字典
        ## magics={}
        actualCode = ''
        newactualCode = ''
        magics = {'codefilename':'',
                  'classname':'',
                  'cflags': [],
                  'ldflags': [],
                  'runinterm': '',
                  'term': 'gnome-terminal',
                  'dlrun': [],
                  'coptions': [],
                  'joptions': [],
                  'onlyrunmagics': '',
                  'package': '',
                  'main': '',
                  'pubclass': '',
                  'runprg': '',
                  'runprgargs': [],
                  'log': '',

                  'repllistpid': [],
                  'replcmdmode': [],
                  'replprompt': [],
                  'replsetip': "\r\n",
                  'replchildpid':"0",

                  'outputtype':'text/plain',
                  'runmode': [],
                  'pid': [],
                  'pidcmd': [],
                  'args': [],
                  'rungdb': []}
        ##扫描源码
        for line in code.splitlines():
            ##扫描源码每行行
            orgline=line
            ## line=self.forcejj2code(line)

            if line==None or line.strip()=='': continue
            if line.strip().startswith('package'):
                qline=self.kobj.replacemany(line,'; ', ';')
                qline=self.kobj.replacemany(qline,' ;', ';')
                qline= qline[:len(qline)-1]
                li=qline.strip().split()
                if(len(li)>1):
                    magics[li[0].strip()] = li[1].strip()
                actualCode += line + '\n'
                continue
            if line.strip().startswith('public'):
                qline = self.kobj.replacemany(line, 's  ', 's ')
                li = qline.strip().split()
                if(len(li) > 2 and li[1].strip() == 'class'):
                    magics[li[0].strip()] = li[1].strip()
                    pubclass = li[2].strip()
                    if pubclass.strip().endswith('{'):
                        pubclass = pubclass[:len(pubclass)-1]
                    magics['pubclass'] = pubclass
                actualCode += line + '\n'
                continue
            if self._is_specialID(line):
                if line.strip()[3:] == "repllistpid":
                    magics['repllistpid'] += ['true']
                    self.repl_listpid()
                    continue
                elif line.strip()[3:] == "onlyrunmagics":
                    magics['onlyrunmagics'] = 'true'
                    continue
                elif line.strip()[3:] == "replcmdmode":
                    magics['replcmdmode'] += ['replcmdmode']
                    continue
                elif line.strip()[3:] == "replprompt":
                    magics['replprompt'] += ['replprompt']
                elif line.strip()[3:] == "runinterm":
                    magics['runinterm'] = 'true'
                    continue
                else:
                    ##通知插件进行预处理
                    ##preprocessor
                    ##通知BOOL型标签插件处理
                    ##on_IBpCodescanning
                    ##调用BOOL标签接口
                    ##%include:../src/_filter2_magics_i1.py
                    ##获得BOOL关键字
                    ##登记Bool型参数和值

                ##获得参数型 关键字和其参数
                findObj= re.search( r':(.*)',line)
                if not findObj or len(findObj.group(0))<2:
                    continue
                key, value = line.strip()[3:].split(":", 1)
                key = key.strip().lower()
                if key in ['ldflags', 'cflags','coptions','joptions']:
                    for flag in value.split():
                        magics[key] += [flag]
                elif key == "runmode":
                    if len(value)>0:
                        magics[key] = value[re.search(r'[^/]',value).start():]
                    else:
                        magics[key] ='real'
                elif key == "replsetip":
                    magics['replsetip'] = value
                elif key == "replchildpid":
                    magics['replchildpid'] = value
                elif key == "pidcmd":
                    magics['pidcmd'] = [value]
                    if len(magics['pidcmd'])>0:
                        findObj= value.split(",",1)
                        if findObj and len(findObj)>1:
                            pid=findObj[0]
                            cmd=findObj[1]
                            self.kobj.send_cmd(pid=pid,cmd=cmd)
                
                elif key == "term":
                    magics['term']=[]
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
                        magics['term'] += [argument.strip('"')]
                elif key == "outputtype":
                    magics[key]=value
                elif key == "log":
                    magics['log'] = value.strip()
                    self.kobj._loglevel= value.strip()
                elif key == "loadurl":
                    url=value
                    if(len(url)>0):
                        line=self.kobj.loadurl(url)
                        actualCode += line + '\n'
                elif key == "runprg":
                    magics['runprg'] = value
                elif key == "runprgargs":
                    ## Split arguments respecting quotes
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
                        magics['runprgargs'] += [argument.strip('"')]
                elif key == "args":
                    ## Split arguments respecting quotes
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
                        magics['args'] += [argument.strip('"')]
                else:
                    pass
                    ##通知插件进行宏标签处理
                    ##登记参数和值
                    ##处理参数和值 ？？？
                    ##合并处理结果
                    ##调用单标签接口
                    ##%include:../src/_filter2_magics_i2.py
            else:
                ## keep lines which did not contain magics
                actualCode += line + '\n'
        newactualCode=actualCode

        bcancel_exec,newcode=self.raise_ICodescan(magics,newactualCode)
        if not bcancel_exec:
            newactualCode=newcode

        ##第二次扫描源代码
            ##扫描源码每行行
            ##清理测试代码 test_begin test_end
            ##清理单行注释 // #
            ##清理多行注释 /* */ ''' """
        ##%include:../src/_filter2_magics_pend.py
        return magics, newactualCode

[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyPython-kernel/plugins/_filter2_magics.py


[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyC-kernel/plugins/_filter2_magics.py


[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyDart-kernel/plugins/_filter2_magics.py


[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyKotlin-kernel/plugins/_filter2_magics.py


[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyGroovy-kernel/plugins/_filter2_magics.py


...

[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyGjs-kernel/plugins/_filter2_magics.py


[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyVala-kernel/plugins/_filter2_magics.py


[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyNodejs-kernel/plugins/_filter2_magics.py


[MyPython] Info:copy to :H:/Jupyter/Myjupyter-kernel/base/../../jupyter-MyBash-kernel/plugins/_filter2_magics.py


[MyPython] Info:file H:/Jupyter/Myjupyter-kernel/base/../src/_filter2_magics.py created successfully


In [ ]:
##%kcmd:list
##%noruncode

In [ ]:
# ------------------------------------------------------------
# calclex.py
#
# tokenizer for a simple expression evaluator for
# numbers and +,-,*,/
# ------------------------------------------------------------
##%overwritefile
##%file:../test/calclex.py
import sys
sys.path.append("/usr/local/lib/python3.9/dist-packages")
from ply import lex as lex

# List of token names.   This is always required
tokens = (
    'spid',
   'test',
   'name',
   'macro',
   'NUMBER',
   'PLUS',
   'MINUS',
   'TIMES',
   'DIVIDE',
   'LPAREN',
   'RPAREN',
)

# Regular expression rules for simple tokens
# t_test    = r'bool'
# t_name    = r'string'
# t_macro    = r'macro'
t_spid=r'abc.*'
t_PLUS    = r'\+'
t_MINUS   = r'-'
t_TIMES   = r'\*'
t_DIVIDE  = r'/'
t_LPAREN  = r'\#\#\%'
t_RPAREN  = r'\n'

# A regular expression rule with some action code
def t_test(t):
    r'\#\#\#\%test'
    # t.value = t.value
    return t
def t_name(t):
    r'\#\#\#\%name+'
    # t.value = t.value
    return t
def t_macro(t):
    r'\#\#\#\%macro+'
    # t.value = t.value
    return t
def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

# Define a rule so we can track line numbers
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

# A string containing ignored characters (spaces and tabs)
t_ignore_COMMENT = r'\#.*'
# Error handling rule
def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)

# Build the lexer
lexer = lex.lex()
# Test it out
data = '''
###%test 'jjjj'
###%name:xxxx
###%macro name value string
'''

# Give the lexer some input
lexer.input(data)

# Tokenize
while True:
    tok = lexer.token()
    if not tok:
        break      # No more input
    print(tok)

In [ ]:
def test():
    'this is test line'
    print(self)
test()